# This script processes the tree related statistics generated by owm_stats.cpp  

1. Generate the file [CloudName].xyz.csv by compiling and running:
    ```
    make bin/stats
    cd scripts
    python3 run_stats.py 
    ```

run_stats.py uses the MinRadius values that result in minimum time without using memoization for OWM= [0.9,0.6,0.2,0.2] and the same for MaxNumber=[512,512,1024,512]

2. Collect the files generated in the cloud directory, with names [CloudName].xyz.csv

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

# object to save the histograms
class histo:
    def __init__(self):
        self.xi=[] # number of points per node
        self.ni=[] # number of nodes with xi points
        self.ri=[] # different radius values
        self.ai=[] # different area values
        self.rni=[] # number of leaf nodes with ri radius
        self.di=[] # different density values
        self.dni=[] # number of leaf nodes with di density
        self.li=[] # level with lni leaf nodes
        self.lni=[] # number of leaf nodes at level i

        self.nleafs=0 # number of leaf nodes
        self.value=0. # value of minRadius/maxNumber
        self.maxh=0 # maximum number of points per node
        self.area=0. # area of each leaf node
        self.density=0. # density of the cloud
        self.odensity=0. # computed density of the cloud
        self.empty_leafs=0 # number of empty leaf nodes

    def compute_params(self):
        # get the number of empty leaf nodes
        self.empty_leafs = self.ni[0]
        # compute the mean area of the cloud
        varea = [x*y for x,y in zip(self.ai, self.rni)]
        self.area = sum(varea)/self.nleafs
        # check that the number of leaf nodes is the same as the sum of the histogram
        assert self.nleafs==sum(self.rni), 'Error: nleaf different from sum(rni)'
        # compute the mean density of the cloud
        vdensity = [x*y for x,y in zip(self.di, self.dni)]
        self.odensity = sum(vdensity)/self.nleafs
        # check
        assert self.nleafs==sum(self.dni), 'Error: nleaf different from sum(dni)'

def get_histogram(lines, pos, newhisto, *args):
    # initilize a dictionay with the new lists
    histo_lists = {arg:[] for arg in args}
    # set a flag to signal the end of the histogram
    flag=False
    # from pos, look for Radius histogram
    for line in lines[pos:]:
        pos += 1
        line = line.split()
        # Check if the first element is a valid number (integer or float)
        try:
            first_element = float(line[0])
            # get the histogram values
            for arg,l in zip(args,line):
                # append the value
                if l.isdigit():
                    histo_lists[arg].append(int(l))
                else:
                    histo_lists[arg].append(float(l))
            flag = True
            continue
        except:
            # Not a valid number (e.g., contains non-numeric characters)
            pass

        # If flag is True, then it is the histogram end
        if flag:
            break

        # set the new lists to the newhisto object
        for arg,value in histo_lists.items():
            setattr(newhisto, arg, value)

    return pos


def readstats(filename):
    f=open(filename)
    lines=f.readlines()
    f.close()

    newhisto = histo()
    # get the number of leaf nodes created
    newhisto.nleafs = int(lines[1].split()[-1]) # line 2
    # get minRadius
    newhisto.value = float(lines[2].split()[1]) # line 3
    # get the maximum number of points per node
    newhisto.maxh = int(lines[7].split()[1]) # line 8
    # get the estimated density of the cloud
    newhisto.density = float(lines[10].split()[-1]) # line 11
    # from line 20 to the line with maxh, read the histogram
    pos=14
    for line in lines[14:]:
        line=line.split()
        newhisto.xi.append(int(line[0]))
        newhisto.ni.append(int(line[1]))
        # stop when the maximum number of points per node is reached
        if int(line[0])==newhisto.maxh:
            break
        pos+=1
    
    pos+=1
    pos = get_histogram(lines, pos, newhisto, 'ri', 'rni', 'ai')
    pos = get_histogram(lines, pos, newhisto, 'di', 'dni')
    pos = get_histogram(lines, pos, newhisto, 'li', 'lni')

    # print('lri:', newhisto.ri)
    # print('len(newhisto.ri):',len(newhisto.ri))
    # print('lrni:', newhisto.rni)
    # print('len(newhisto.rni):',len(newhisto.rni))
    # print('lai:', newhisto.ai)
    # print('len(newhisto.ai):',len(newhisto.ai))
    # print('ldi:', newhisto.di)
    # print('len(newhisto.di):',len(newhisto.di))
    # print('ldni:', newhisto.dni)
    # print('len(newhisto.dni):',len(newhisto.dni))
    # print('lli:', newhisto.li)
    # print('len(newhisto.li):',len(newhisto.li))
    # print('llni:', newhisto.lni)
    # print('len(newhisto.lni):',len(newhisto.lni))

    # compute hist parameters
    newhisto.compute_params()

    return newhisto


In [ ]:
# get the hostname
hostname = os.popen("hostname").read().strip()
# check minRadius and maxNumber directories
maxNum_dir = f'{hostname}/treeStatistics/MaxNumHisto-{hostname}'
minRad_dir = f'{hostname}/treeStatistics/MinRadHisto-{hostname}'

clouds=['Alcoy','Arzua','BrionF','BrionU']
minRadius = [2.0, 0.9, 0.6, 0.1] # this values match those used in run_stats.py
best_minRadius = [0.9,0.6,0.1,0.1] # best minRadius for each cloud

all_histos = {}
# all_nph=[]
for cloud in clouds:
    all_histos[cloud] = {}
    for mR in minRadius:
        print('Cloud:',cloud)
        all_histos[cloud][mR] = readstats(f"{minRad_dir}/{cloud}H_hist_mR{mR}.csv")
        print('minRadius:',all_histos[cloud][mR].value, end=', ')
        print('nleafs:',all_histos[cloud][mR].nleafs)
    print()

# For different values of MinRadius, is there a difference between the observed densities?

In [ ]:
for cloud in clouds:
    print('Cloud: ', cloud)
    for mR in all_histos[cloud]:
        print(f'mR={mR}: {all_histos[cloud][mR].odensity:.2f}', end=', ')
    print()

In [ ]:
points_per_leaf = 34
for cloud,mR in zip(clouds,best_minRadius):
    print('Density:',all_histos[cloud][mR].density)
    print(f'Best minRadius: {all_histos[cloud][mR].value}')
    print(f'Estimated minRadius: {np.sqrt(points_per_leaf/all_histos[cloud][mR].density)/2}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Datos de los puntos
x_data = np.array([0.9, 0.6, 0.1, 0.1]) # minRadius
y_data = np.array([7.57, 40.7, 119.7, 122.11]) # Densidad media

# Función exponencial: y = ae^(bx)
def exponential_func(x, a, b):
    return a * np.exp(b * x)
# Función cúbica: y = ax^3 + bx^2 + cx + d
def cubic_func(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d
# Función cuadrática: y = ax^2 + bx + c
def quadratic_func(x, a, b, c):
    return a * x**2 + b * x + c
# linear function
def linear_func(x, a, b):
    return a*x + b

# Define la función y = 64/(x*2)^2
def func2(x):
    return 64 / (x*2)**2

# Aproximación de parámetros para las curvas
exp_params, _ = curve_fit(exponential_func, x_data, y_data, p0=[122.11, -2.3])
cubic_params, _ = curve_fit(cubic_func, x_data, y_data, p0=[-135.4, 370.6, -222.9, 40.7])
quadratic_params, _ = curve_fit(quadratic_func, x_data, y_data, p0=[119.7, -239.4, 122.11])
linear_params, _ = curve_fit(linear_func, x_data, y_data, p0=[1,1])
print(linear_params)

# Valores estimados para los puntos dados
x_vals = np.linspace(0, 1, 100)
y_exp = exponential_func(x_vals, *exp_params)
y_cubic = cubic_func(x_vals, *cubic_params)
y_quadratic = quadratic_func(x_vals, *quadratic_params)
y_linear = linear_func(x_vals, *linear_params)

y_orig = func2(x_vals)

# Gráfico
plt.figure(figsize=(8, 6))
plt.scatter(x_data, y_data, label='Datos', color='red')
plt.plot(x_vals, y_exp, label='Curva Exponencial', linestyle='--')
plt.plot(x_vals, y_cubic, label='Curva Cúbica', linestyle='-.')
plt.plot(x_vals, y_quadratic, label='Curva Cuadrática', linestyle=':')
plt.plot(x_vals, y_linear, label='Curva Lineal', linestyle='-', color='purple')
plt.plot(x_vals, y_orig, label="y = 64/(x*2)^2", color="green")
plt.xlabel('minRadius [m]')
plt.ylabel('Densidad media [puntos/m^2]')
plt.ylim([0, 150])
plt.title('Curvas A8')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# import palette
from seaborn.palettes import color_palette

def plot_histo2(all_histo, bestvalues, nbins=512, is_maxNumber=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 18
    yticksfs = 18
    legendfs = 12
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    clouds=['Alcoy','Arzua','BrionF','BrionU']

    # get the color palette and supose that all clouds have the same number of values
    colors = color_palette('deep', n_colors=len(all_histo[clouds[0]].keys()))

    strategy = 'MaxNumber' if is_maxNumber else 'MinRadius'

    # dict to save mode and mean values
    stats = {cloud:{} for cloud in clouds}
    
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(15, 5), constrained_layout=True)
    for i,cloud in enumerate(clouds):
        for j,value in enumerate(all_histo[cloud].keys()):

            # represent only the first values withou 0 values
            limit = min(nbins, len(all_histo[cloud][value].ni))
            # get the histogram values and normalize them
            
            # drop 0 and the last values
            chunk = all_histo[cloud][value].ni[1:limit]
            # chunk = [x/all_histos[name][value].nleafs for x in all_histo[name][value].ni[1:limit]]
            chunkx = np.arange(0,limit)

            # add BEST if it is the best minRadius
            if value == bestvalues[i]:
                # get the histogram plot at the front of the figure
                axs[i].stairs(chunk, chunkx, fill=True, label=f'{strategy}: {value} (Opt.)', color=colors[j], zorder=10)
                # compute the histogram mean discarding 0 values
                mean=0
                for k in chunkx[:-1]:
                    mean+=k*chunk[k]
                mean=mean/np.sum(chunk)
                print('mean:',mean)
                # plot a vertical line at the mean
                # axs[i].axvline(mean, color='r', linestyle='dashed', linewidth=1)
                stats[cloud]['mean'] = mean

                # compute the histogram mode discarding 0 values
                mode=0
                maxh=0
                for k in chunkx[:-1]:
                    if chunk[k] > maxh:
                        maxh=chunk[k]
                        mode=k
                print('mode:',mode)
                # plot a vertical line at the mode
                # axs[i].axvline(mode, color='g', linestyle='dashed', linewidth=1)     
                stats[cloud]['mode'] = mode   
                stats[cloud]['y_mode'] = maxh
            else:
                try:
                    axs[i].stairs(chunk, chunkx, fill=True, label=f'{strategy}: {value}', color=colors[j], alpha=0.5)
                except:
                    print('Error: ',cloud,value)
                    print('pos: ',j)
                            
        axs[i].set_title(cloud,fontsize=16)
        axs[i].set_xlabel('Number of points', fontsize=xlabelfs)
        axs[i].set_xlim(-32,min(nbins, 2048))
        # axs[i].set_ylim(0,512)
        # log y axis
        axs[i].set_yscale('log')

        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
        axs[i].legend(loc='best', fontsize=legendfs)
    
    fig.suptitle(f"{strategy}: Histogram of number of leaf-nodes with x points",  fontweight='bold', fontsize=titlefs)
    
    axs[0].set_ylabel('Number of leaf-nodes', fontsize=ylabelfs)
    pp = PdfPages(f"{hostname}/Histogram_num_points_{strategy}.pdf")
    pp.savefig(fig)
    pp.close()

    return stats


mR_stats = plot_histo2(all_histos, best_minRadius, nbins=1024)

In [ ]:
#number of empty leaf-nodes
empty_leaf_nodes=[]
for cloud,mR in zip(all_histos.keys(), best_minRadius):
    print('Cloud: ', cloud, end='')
    print(f'. Number of empty leaf-nodes: {all_histos[cloud][mR].empty_leafs}')  
    empty_leaf_nodes.append(all_histos[cloud][mR].empty_leafs)

In [ ]:
#number of leaf-nodes with more than 512 points
for cloud,mR in zip(all_histos.keys(), best_minRadius):
    print('Cloud: ', cloud, end='')
    # get the position of xi >= 512
    try:
        pos = np.where(np.array(all_histos[cloud][mR].xi)>=512)[0][0]
        print('. Number of leaf-nodes with more than 512 points: {}'.format(np.sum(all_histos[cloud][mR].ni[pos:])))
    except IndexError:
        print('. Number of leaf-nodes with more than 512 points: 0')

# Compute the histograms for maxNumber results

In [ ]:
all_histos_mn = {}
maxNumber=[1024,512,256,128,64,32] # this values match those used in run_stats.py
best_maxNumber=[128,256,64,64] # best maxNumber for each cloud
for cloud in clouds:
    all_histos_mn[cloud] = {}
    for mN in maxNumber:
        print('Cloud:',cloud)
        all_histos_mn[cloud][mN] = readstats(f"{maxNum_dir}/{cloud}H_hist_mN{mN}.csv")
        print('minRadius:',all_histos_mn[cloud][mN].value, end=', ')
        print('nleafs:',all_histos_mn[cloud][mN].nleafs)
    print()

In [ ]:
mN_stats = plot_histo2(all_histos_mn, best_maxNumber, nbins=1024, is_maxNumber=True)

In [ ]:
mR_stats

In [ ]:
mN_stats

In [ ]:
#number of empty leaf-nodes
for cloud,mN in zip(all_histos_mn.keys(), best_maxNumber):
    print('Cloud: ', cloud, end='')
    print('. Number of empty leaf-nodes: {}'.format(all_histos_mn[cloud][mN].empty_leafs)) 
    empty_leaf_nodes.append(all_histos_mn[cloud][mN].empty_leafs)

#number of leaf-nodes with more than 512 points
for cloud,mN in zip(all_histos_mn.keys(), best_maxNumber):
    print('Cloud: ', cloud, end='')
    try:
        pos = np.where(np.array(all_histos_mn[cloud][mN].xi)>=512)[0][0]
        print('. Number of leaf-nodes with more than 512 points: {}'.format(np.sum(all_histos_mn[cloud][mN].ni[pos:])))
    except IndexError:
        print('. Number of leaf-nodes with more than 512 points: 0')

# Build LaTeX table

In [ ]:
# save the min and max levels
min_level = 1000
max_level = 0

level_mR = {}
for cloud,mR in zip(clouds,best_minRadius):
    # level_mR[mR] = {}
    level_mR[cloud] = {}
    for i in range(len(all_histos[cloud][mR].li)):
        lev = all_histos[cloud][mR].li[i]
        level_mR[cloud][lev] = all_histos[cloud][mR].lni[i]
        if lev < min_level:
            min_level = lev
        if lev > max_level:
            max_level = lev
level_mN = {}
for cloud,mN in zip(clouds,best_maxNumber):
    # level_mN[mN] = {}
    level_mN[cloud] = {}
    for i in range(len(all_histos_mn[cloud][mN].li)):
        lev = all_histos_mn[cloud][mN].li[i]
        level_mN[cloud][lev] = all_histos_mn[cloud][mN].lni[i]
        if lev < min_level:
            min_level = lev
        if lev > max_level:
            max_level = lev

print(f"Min level: {min_level}")
print(f"Max level: {max_level}")

In [ ]:
level_mR

In [ ]:
level_mN

In [ ]:
def print_level(n):
    if n < 1e6:
        print(" & {}".format(n), end='')
    else:
        # print in scientific notation
        print(" & {:.1e}".format(n), end='')

print("\\begin{tabular}{|c|cccc|cccc|} \hline")
print("Strategy & \multicolumn{4}{c|}{MinRadius} & \multicolumn{4}{c|}{MaxNumber} \\\\ \hline")  
print("Cloud & Alcoy & Arzua & BrionF & BrionU & Alcoy & Arzua & BrionF & BrionU \\\\ \hline")
# row with the optimal values
print("\parbox[c][3em][c]{4.5em}{\centering Optimal Value}", end='')
for val in best_minRadius:
    print(" & {}".format(val),end='')
for val in best_maxNumber:
    print(" & {}".format(val),end='')
print(" \\\\ \hline")
# mode
print("Mode", end='')
for cloud in clouds:
    print(" & {}".format(mR_stats[cloud]['mode']), end='')
for cloud in clouds:
    print(" & {}".format(mN_stats[cloud]['mode']), end='')
print(" \\\\ \hline")
# observed density
print("\parbox[c][3em][c]{4.5em}{\centering Observed Density}", end='')
for cloud,mR in zip(clouds,best_minRadius):
    print(" & {:.1f}".format(all_histos[cloud][mR].odensity), end='')
for cloud,mN in zip(clouds,best_maxNumber):
    print(" & {:.1f}".format(all_histos_mn[cloud][mN].odensity), end='')
print(" \\\\ \hline")
# area
print("Area", end='')
for cloud,mR in zip(clouds,best_minRadius):
    print(" & {:.2f}".format(all_histos[cloud][mR].area), end='')
for cloud,mN in zip(clouds,best_maxNumber):
    print(" & {:.2f}".format(all_histos_mn[cloud][mN].area), end='')
print(" \\\\ \hline")
# number of empty leafs
print("\parbox[c][3em][c]{4.5em}{\centering \# empty leaf-nodes}", end='')
for i in empty_leaf_nodes:
    print(" & {}".format(i), end='')
print(" \\\\ \hline")
print("\hline", end='\n\n')

print("\parbox[c][3em][c]{4.5em}{\centering Tree Level} & \multicolumn{8}{c|}{\centering \# leaf-nodes}  \\\\ \hline")
# print levels
for lev in range(min_level, max_level+1):
    print(lev,end='')
    for cloud in clouds:
        if level_mR[cloud].get(lev) is None:
            print(" & 0",end='')
        else:
            print_level(level_mR[cloud][lev])
    for cloud in clouds:
        if level_mN[cloud].get(lev) is None:
            print(" & 0",end='')
        else:
            print_level(level_mN[cloud][lev])
    print(" \\\\ \hline")

# number of leaf nodes
print("Total",end='')
for cloud,mR in zip(clouds,best_minRadius):
    # print(" & {}".format(all_histos[cloud][mR].nleafs),end='')
    print_level(all_histos[cloud][mR].nleafs)
for cloud,mN in zip(clouds,best_maxNumber):
    # print(" & {}".format(all_histos_mn[cloud][mN].nleafs),end='')
    print_level(all_histos_mn[cloud][mN].nleafs)
print(" \\\\ \hline")
print("\end{tabular}")